## Import Librairies

In [7]:
import os 
import random
import pandas as pd
import numpy as np
import ast
import datetime
import pickle

import matplotlib.pyplot as plt

from functools import partial

from tqdm import tqdm 

tqdm.pandas()

In [8]:
import torch 
from keras_preprocessing.sequence import pad_sequences

In [9]:
try:
    from transformers import GPT2Tokenizer, GPT2LMHeadModel
except:
    !pip install transformers
    from transformers import GPT2Tokenizer, GPT2LMHeadModel

## Import GPT-2

In [10]:
# Load pretrained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("asi/gpt-fr-cased-small")
tokenizer = GPT2Tokenizer.from_pretrained("asi/gpt-fr-cased-small")

## Set up drive

In [11]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [14]:
DRIVEWORKSPACE_PATH = "/content/gdrive/Shareddrives/# Zone de Code #/AIRapFR/"
DRIVEWORKSPACE_PATH = ""
DATASETS_DIR_PATH = f"{DRIVEWORKSPACE_PATH}data/datasets/"

OUTPUTS_DIR_PATH = f"{DRIVEWORKSPACE_PATH}ai/outputs/"  #txt files

MODELS_DIR_PATH = f"{DRIVEWORKSPACE_PATH}ai/models/"
LSTM_MODELS_DIR_PATH = f"{MODELS_DIR_PATH}lstm/"

for p in [DATASETS_DIR_PATH, MODELS_DIR_PATH, LSTM_MODELS_DIR_PATH, OUTPUTS_DIR_PATH]:
    if not os.path.exists(p[:-2]):
        raise Exception(f"{p} not exists")

Exception: ignored

In [ ]:
#import sys
#sys.path.append(DRIVEWORKSPACE_PATH)

In [ ]:
from utils import create_dir
from data.load_corpus import CorpusDataManager

#cdm = CorpusDataManager()

## Text Data

In [ ]:
ARTIST_ID = 1273

### Rap Corpus

In [ ]:
#df_corpus_rap = cdm.get_full_df_lyrics_corpus(preprocessed=True, tokenised_output=False, punct_removal=True, ignore_artist_ids=[1273])
#texts_rap = df_corpus_rap["lyrics"].tolist()
with open(f"{DATASETS_DIR_PATH}texts_rap_corpus.pkl","rb") as f:
    texts_rap = pickle.load(f)

In [ ]:
#with open(f"{DATASETS_DIR_PATH}/texts_artist_{ARTIST_ID}.pkl") as f:
#    texts_artist = pickle.load(f)

In [ ]:
texts_rap[0]

In [ ]:
#file_path = f"{DATASETS_DIR_PATH}/texts_rap_corpus.pkl"
#with open(file_path, "wb") as f:
#    pickle.dump(texts_rap, f)

In [ ]:
#del df_corpus_rap

### Artist Corpus

In [ ]:
#df_artist = cdm.get_df_lyrics_preprocessed_by_genius_id(ARTIST_ID)
#texts_artist = df_artist["lyrics"].tolist()

with open(f"{DATASETS_DIR_PATH}/texts_artist_{ARTIST_ID}.pkl","rb") as f:
    texts_artist = pickle.load(f)
    del texts_artist

In [ ]:
#max_length_corpus_rap = max(max([len(text) for text in texts_artist]),max_length_corpus_rap)
#max_length_corpus_rap 

In [ ]:
"""file_path = f"{DATASETS_DIR_PATH}/texts_artist_{ARTIST_ID}.pkl"
with open(file_path, "wb") as f:
    pickle.dump(texts_artist, f)

"""

In [ ]:
#del df_artist

### Preprocessing

In [ ]:
texts_rap = texts_rap[:5000]
# Encode texts as input sequences
input_ids = [tokenizer.encode(text, return_tensors="pt") for text in tqdm(texts_rap)]

In [ ]:
len(input_ids)

In [ ]:
lens_input_ids = [input_id.size() for input_id in input_ids]
plt.hist(lens_input_ids, bins=100)

In [ ]:
max_length_corpus_rap = np.max(lens_input_ids)
mean_length_corpus_rap = int(np.mean(lens_input_ids))
q0_7_length_corpus_rap = int(np.quantile(lens_input_ids, 0.6))
max_length_corpus_rap, mean_length_corpus_rap, q0_7_length_corpus_rap

In [ ]:
MAX_LENGTH = 50
MAX_LENGTH

In [ ]:
# Pad sequences to have the same length
input_ids = pad_sequences(input_ids, padding="post", maxlen=MAX_LENGTH)

# Prepare attention masks to avoid attention on padding
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

# Create training tensors
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(input_ids)
train_masks = torch.tensor(attention_masks)

## Fine Tuning

In [ ]:
# Initialize a TensorBoard writer
writer_tensorboardx = SummaryWriter()

In [ ]:
%tensorboard --logdir runs

In [ ]:
LEARNING_RATE = 0.0001
EPOCHS = 50

In [ ]:


# Set the model in training mode
model.train()

# Fine-tune the model on your custom dataset
for epoch in range(EPOCHS):
  
    # Define the loss function and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Reset the gradients to zero
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(train_inputs, attention_mask=train_masks)
    
    # Calculate the loss
    loss = loss_fn(outputs[0][:,:-1].contiguous().view(-1, outputs.size(-1)),
                   train_labels[:,1:].contiguous().view(-1))
    
    # Backward pass
    loss.backward()
    
    # Update the model parameters
    optimizer.step()

    # Write the loss to TensorBoard
    #writer_tensorboardx.add_scalar('Loss/train', loss.item(), epoch)
    
    # Print the training loss for each epoch
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {loss.item()}")